# Evaluate fine-tuned sensationalist detector model

#### Load sensationalist detector for inference

In [1]:
from unsloth import FastLanguageModel
max_seq_length = 2048 
dtype = None 
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "llama-3-8b-bnb-4bit-sft-sensationalist-detector-v2_model",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-07-01 21:42:18.285966: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-01 21:42:18.310999: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-01 21:42:18.710603: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.754 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Format testset as the instructions from the training set 

In [4]:
prompt = """ 
Below you will find the title, summary, and body of a news article. 
Your task is to analyze these components and classify whether the article is sensationalist or not.

Sensationalist is defined as: "presenting information in a way that is intended to provoke public interest, excitement, or anxiety, at the expense of accuracy."

### Article information:
    Title: {}
    Subheading: {}
    Body: {}
    is this article false?: {}
    
### is this article sensationalist?:
{}
"""
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    
    title = examples["Titular"]
    summary = examples["Copete"]
    body      = examples["Cuerpo"]
    is_sensationalist      = examples["Amarillismo"]
    is_false = examples["Falsa"]
    texts = []
    for title, summary, body, is_false, is_sensationalist in zip(title, summary, body, is_false, is_sensationalist):
        # Must add EOS_TOKEN
        text = prompt.format(title, summary, body, is_false, is_sensationalist) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

### Define functions for inference, load testset and get predictions

In [5]:

def inference(article_data, model, tokenizer):
    
    inputs = tokenizer(
    [
        prompt.format(
            article_data["Titular"], # titular
            article_data["Copete"], # summary
            article_data["Cuerpo"],# body
            article_data["Falsa"], # is false
    ""
    )], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    return tokenizer.batch_decode(outputs)

import pandas as pd
def load_dataset(dataset_path):
    dataset = pd.read_json(dataset_path, orient='index',  encoding='latin1')
    return dataset

def text_generated2label(text_generated):
    is_sensationalist_answer = text_generated[text_generated.find("### is this article sensationalist?:\n") + len("### is this article sensationalist?:\n"):]
    if("No Amarillista" in is_sensationalist_answer):
        return 0
    else:
        return 1

#### Load testset

In [18]:
testset = pd.read_csv("data/testset_realnews.csv", encoding='latin1')
testset["Amarillismo_binary"] = testset["Amarillismo"].apply(lambda x: 0 if x == "No Amarillista" else 1)

Make predictions for the test set

In [17]:
def make_predictions(model_name, model, tokenizer, dataset):
    predictions = pd.DataFrame(columns=["Id","Predicted_label", "True_label", "Generated_text"])
    for i in range(len(dataset)):
        article_data = dataset.iloc[i]
        text_generated = inference(article_data, model, tokenizer)
        label = text_generated2label(text_generated[0])
        predictions.loc[len(predictions)] = [article_data["Id"], label, article_data["Amarillismo_binary"], text_generated[0]]

    predictions.to_csv(f'predictions_{model_name}.csv', index=False)
    return predictions

In [6]:
predictions_llama_sft_sensationalist = make_predictions("llama-3-8b-bnb-4bit-sft-sensationalist-detector-v2-testset", model, tokenizer, testset)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

### Analyse predictions 

In [19]:
from sklearn.metrics import classification_report, balanced_accuracy_score

def generate_report(predictions):
    report = classification_report(predictions["True_label"], predictions["Predicted_label"], labels=[0,1], target_names=["No Amarillista", "Amarillista"], output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    
    balanced_accuracy = balanced_accuracy_score(predictions["True_label"], predictions["Predicted_label"])
    print("Balanced accuracy: " + str(balanced_accuracy))
    return report_df, balanced_accuracy

Report for sft model on testset

In [8]:
report, balanced_accuracy_score = generate_report(predictions_llama_sft_sensationalist)
report

Balanced accuracy: 0.6286764705882353


,precision,recall,f1-score,support
No Amarillista,0.818182,0.375000,0.514286,24.000000
Amarillista,0.500000,0.882353,0.638298,17.000000
accuracy,0.585366,0.585366,0.585366,0.585366
macro avg,0.659091,0.628676,0.576292,41.000000
weighted avg,0.686253,0.585366,0.565705,41.000000


Eval sft model on trainset

In [20]:
trainset = load_dataset("data/trainset.json")
trainset["Id"] = trainset.index
trainset["Amarillismo_binary"] = trainset["Amarillismo"].apply(lambda x: 0 if x == "No Amarillista" else 1)

In [9]:
predictions_llama_sft_sensationalist_trainset = make_predictions("llama-3-8b-bnb-4bit-sft-sensationalist-detector-v2-trainset", model, tokenizer, trainset)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [15]:
report, balanced_accuracy_score = generate_report(predictions_llama_sft_sensationalist_trainset)
report

Balanced accuracy: 0.905940594059406


,precision,recall,f1-score,support
No Amarillista,1.000000,0.811881,0.896175,101.000000
Amarillista,0.841667,1.000000,0.914027,101.000000
accuracy,0.905941,0.905941,0.905941,0.905941
macro avg,0.920833,0.905941,0.905101,202.000000
weighted avg,0.920833,0.905941,0.905101,202.000000


#### hallucinations

In [21]:
def detect_hallucinations(predictions):
    
    hallucinations_responses_idx = [] 
    for i in range(len(predictions)):
        response = predictions['Generated_text'][i]
        response_is_sensationalist = response[response.find("### is this article sensationalist?:\n") + len("### is this article sensationalist?:\n"):]
        
        if "No Amarillista" not in response_is_sensationalist  and "Amarillista" not in response_is_sensationalist:
            print(response_is_sensationalist)
            hallucinations_responses_idx.append(predictions.iloc[i].name)
            
    print(f"Number of hallucinations {len(hallucinations_responses_idx)} out of {len(predictions)} predictions.")
    return hallucinations_responses_idx

In [60]:
# Detect hallucinations in the predictions in testset
detect_hallucinations(predictions_llama_sft_sensationalist)

# Detect hallucinations in the predictions in trainset
detect_hallucinations(predictions_llama_sft_sensationalist_trainset)

Number of hallucinations 0 out of 41 predictions.
Number of hallucinations 0 out of 202 predictions.


[]

## Eval base model without fine-tuning 

In [1]:
from unsloth import FastLanguageModel
max_seq_length = 2048 
dtype = None 
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-07-01 21:57:04.035738: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-01 21:57:04.061084: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-01 21:57:04.465294: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.754 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Redefine prompt to get a yes or no answer

In [11]:
prompt = """ 
Below you will find the title, summary, and body of a news article. 
Your task is to analyze these components and classify whether the article is sensationalist or not.

Sensationalist is defined as: "presenting information in a way that is intended to provoke public interest, excitement, or anxiety, at the expense of accuracy."

### Article information:
    Title: {}
    Subheading: {}
    Body: {}
    is this article false?: {}
    
### is this article sensationalist? (Answer with Yes or No):
{}
"""

In [15]:
#redefining the function to work with the new prompt format
def text_generated2label(text_generated):
    is_sensationalist_question = "### is this article sensationalist? (Answer with Yes or No):\n"
    is_sensationalist_answer = text_generated[text_generated.find(is_sensationalist_question) + len(is_sensationalist_question):]
    if("Yes" in is_sensationalist_answer):
        return 1
    else:
        return 0

In [22]:
predictions_llama_base_testset = make_predictions("llama-3-8b-bnb-4bit-testset", model, tokenizer, testset)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [25]:
report, accuracy = generate_report(predictions_llama_base_testset)
report

Balanced accuracy: 0.4276960784313726


,precision,recall,f1-score,support
No Amarillista,0.454545,0.208333,0.285714,24.000000
Amarillista,0.366667,0.647059,0.468085,17.000000
accuracy,0.390244,0.390244,0.390244,0.390244
macro avg,0.410606,0.427696,0.376900,41.000000
weighted avg,0.418108,0.390244,0.361331,41.000000


In [38]:
def detect_hallucinations_base_model(predictions):
        
    hallucinations_responses_idx = [] 
    for i in range(len(predictions)):
        text_generated = predictions['Generated_text'][i]
        is_sensationalist_question = "### is this article sensationalist? (Answer with Yes or No):\n"
        is_sensationalist_answer = text_generated[text_generated.find(is_sensationalist_question) + len(is_sensationalist_question):]
        
        if("Yes" not in is_sensationalist_answer and "No" not in is_sensationalist_answer):
            hallucinations_responses_idx.append(predictions.iloc[i].name)            
    print(f"Number of hallucinations {len(hallucinations_responses_idx)} out of {len(predictions)} predictions.")
    return hallucinations_responses_idx

In [40]:
hallucinations_idx = detect_hallucinations_base_model(predictions_llama_base_testset)

Number of hallucinations 39 out of 41 predictions.


In [41]:
print(predictions_llama_base_testset.iloc[hallucinations_idx[4]]["Generated_text"])

<|begin_of_text|> 
Below you will find the title, summary, and body of a news article. 
Your task is to analyze these components and classify whether the article is sensationalist or not.

Sensationalist is defined as: "presenting information in a way that is intended to provoke public interest, excitement, or anxiety, at the expense of accuracy."

### Article information:
    Title: Impresionantes hallazgos en el espacio y en las aguas del golfo de MÃ©xico
    Subheading: nan
    Body: Impresionantes hallazgos en el espacio y en las aguas del golfo de MÃ©xico
 La NASA descubriÃ³ una reconexiÃ³n magnÃ©tica que tendrÃ¡ relevancia en futuros estudios sobre el Sol. AdemÃ¡s, la Oficina Nacional de AdministraciÃ³n OceÃ¡nica y AtmosfÃ©rica de Estados Unidos explorÃ³ las aguas del golfo de MÃ©xico y descubriÃ³ nuevas especies.
    is this article false?: Verdadera
    
### is this article sensationalist? (Answer with Yes or No):

Yes
<|end_of_text|>
